# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

#df.head()

df_binary_encoded = pd.get_dummies(df, columns = [
    'initial_list_status',
    'home_ownership',
    'verification_status',
    'issue_d',
    'pymnt_plan',
    'next_pymnt_d',
    'application_type',
    'hardship_flag',
    'debt_settlement_flag'])
df_binary_encoded.head(20)
#df_binary_encoded = pd.get_dummies(df)
#df_binary_encoded.head()
#df_binary_encoded.columns



,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,pymnt_plan_n,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,0,0,1,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,0,0,1,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,0,0,1,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,0,0,1,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,0,0,1,1,0,1,1,0,1,1
5,25000.0,0.1797,634.43,90000.0,low_risk,18.49,1.0,1.0,14.0,0.0,...,0,0,1,1,0,1,1,0,1,1
6,20400.0,0.2000,540.48,51315.0,low_risk,25.63,0.0,2.0,8.0,1.0,...,0,0,1,1,0,1,1,0,1,1
7,10000.0,0.1557,349.46,66000.0,low_risk,9.00,0.0,0.0,11.0,0.0,...,0,0,1,1,0,1,1,0,1,1
8,14000.0,0.0881,443.96,45000.0,low_risk,10.24,0.0,1.0,9.0,0.0,...,0,0,1,1,0,1,1,0,1,1
9,10000.0,0.1474,345.39,63295.0,low_risk,39.34,0.0,0.0,27.0,0.0,...,0,0,1,1,0,1,1,0,1,1


# Split the Data into Training and Testing

In [31]:
# Create our features
X = df_binary_encoded.drop(columns = 'loan_status', axis =1)

# Create our target
y = df['loan_status']


In [32]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,pymnt_plan_n,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.371696,0.451066,0.177238,1.0,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.483261,0.497603,0.381873,0.0,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,0.000000,0.000000,1.0,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,1.000000,0.000000,1.0,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [33]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify = y)
#reshape strings to integers 
X_train.shape

(51612, 95)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [35]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'low_risk': 51352, 'high_risk': 51352})

In [36]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [37]:

y_pred = model.predict(X_test)
result = pd.DataFrame({"Predictions": y_pred, "Actual": y_test}).reset_index(drop=True)
result.head(10)

,Predictions,Actual
0,high_risk,low_risk
1,high_risk,low_risk
2,high_risk,low_risk
3,high_risk,low_risk
4,high_risk,low_risk
5,high_risk,low_risk
6,high_risk,low_risk
7,low_risk,low_risk
8,low_risk,low_risk
9,low_risk,low_risk


In [38]:
# Calculated the balanced accuracy score
from sklearn.metrics import accuracy_score
acc_score = (accuracy_score(y_test, y_pred))
print(acc_score)

0.6696890438825923


In [39]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report 
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual High-Risk", "Actual Low-Risk"], columns = ["Predicted High-Risk", 
                                                                                  "Predicted Low-Risk"])
cm_df

,Predicted High-Risk,Predicted Low-Risk
Actual High-Risk,54,33
Actual Low-Risk,5650,11468


In [40]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced 
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.62      0.67      0.02      0.64      0.41        87
   low_risk       1.00      0.67      0.62      0.80      0.64      0.42     17118

avg / total       0.99      0.67      0.62      0.80      0.64      0.42     17205



### SMOTE Oversampling

In [41]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resample2, y_resample2 = SMOTE(random_state = 1, sampling_strategy ='auto').fit_resample(X_train, y_train)

In [42]:
# Train the Logistic Regression model using the resampled data
model_smote = LogisticRegression(random_state=1)
model_smote.fit(X_resample2, y_resample2)
y_pred_smote = model_smote.predict(X_test)
y_pred_smote

array(['high_risk', 'high_risk', 'high_risk', ..., 'low_risk',
       'high_risk', 'low_risk'], dtype=object)

In [43]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
bal_acc_score = balanced_accuracy_score(y_test, y_pred_smote)
bal_acc_score

0.619302730338301

In [45]:
# Display the confusion matrix
cm_smote = confusion_matrix(y_test, y_pred_smote)
cm_smote_df = pd.DataFrame(cm_smote, index = ["Actual High-Risk", "Actual Low-Risk"], columns = ["Predicted High-Risk", 
                                                                                  "Predicted Low-Risk"])
cm_smote_df

,Predicted High-Risk,Predicted Low-Risk
Actual High-Risk,52,35
Actual Low-Risk,6147,10971


In [46]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_smote))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.60      0.64      0.02      0.62      0.38        87
   low_risk       1.00      0.64      0.60      0.78      0.62      0.38     17118

avg / total       0.99      0.64      0.60      0.78      0.62      0.38     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [47]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resample3, y_resample3 = cc.fit_resample(X_train, y_train)
Counter(y_resample3)

Counter({'high_risk': 260, 'low_risk': 260})

In [48]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model_cc = LogisticRegression(random_state=1)
model_cc.fit(X_resample3, y_resample3)

y_pred_cc = model_cc.predict(X_test)
y_pred_cc


array(['high_risk', 'high_risk', 'high_risk', ..., 'high_risk',
       'high_risk', 'high_risk'], dtype=object)

In [49]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
cc_acc_score = balanced_accuracy_score(y_test, y_pred_cc)
cc_acc_score

0.5102725100821478

In [50]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm_cc = confusion_matrix(y_test, y_pred_cc)
cm_cc_df = pd.DataFrame(cm_cc, index = ["Actual High-Risk", "Actual Low-Risk"], columns = ["Predicted High-Risk", 
                                                                                  "Predicted Low-Risk"])
cm_cc_df

,Predicted High-Risk,Predicted Low-Risk
Actual High-Risk,51,36
Actual Low-Risk,9683,7435


In [51]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced 
print(classification_report_imbalanced(y_test, y_pred_cc))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.59      0.43      0.01      0.50      0.26        87
   low_risk       1.00      0.43      0.59      0.60      0.50      0.25     17118

avg / total       0.99      0.44      0.59      0.60      0.50      0.25     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [52]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN 
smote_enn = SMOTEENN(random_state=1)
X_resample4, y_resample4 = smote_enn.fit_resample(X_train, y_train)
Counter(y_resample4)

Counter({'high_risk': 51351, 'low_risk': 46389})

In [53]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model_senn = LogisticRegression(random_state=1)
model_senn.fit(X_resample4, y_resample4)

y_pred_senn = model_senn.predict(X_test)

In [54]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score 
senn_acc_score = balanced_accuracy_score(y_test, y_pred_senn)
senn_acc_score

0.6314637546281188

In [55]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm_senn = confusion_matrix(y_test, y_pred_senn)
cm_senn_df = pd.DataFrame(cm_senn, index = ["Actual High-Risk", "Actual Low-Risk"], columns = ["Predicted High-Risk", 
                                                                                  "Predicted Low-Risk"])
cm_senn_df

,Predicted High-Risk,Predicted Low-Risk
Actual High-Risk,63,24
Actual Low-Risk,7895,9223


In [56]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_senn))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.72      0.54      0.02      0.62      0.40        87
   low_risk       1.00      0.54      0.72      0.70      0.62      0.38     17118

avg / total       0.99      0.54      0.72      0.70      0.62      0.38     17205



In [30]:
#by Jing Tang
#Columbia University Data Analytics Boot Camp